<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_200DMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-12-12 18:33:45


In [8]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [9]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

mypf = mypf[(mypf['Criteria'] == 'X40') | (mypf['Criteria'] == 'V40N')]
#mypf = mypf[mypf['TFrame'] == 'D']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [10]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import pandas_ta as ta

In [11]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [12]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      #mid_window = 50
      long_window = 200
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_DMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_DMA'])*100/stock_df['200_DMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      #df_prec_dev = df_prec_dev.append(tmp)
      df_prec_dev = pd.concat([df_prec_dev, tmp])
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):

    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] < 30) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev = df_prec_dev[df_prec_dev['Signal'] != 0]

    return df_prec_dev.tail(1)

def get_daily_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_tmp['InFolio'] = mypf[(mypf['Symbol']) == stock_id.upper()]['InFolio'].values[0]
        #df_buy_sell = df_buy_sell.append(df_tmp)
        df_buy_sell = pd.concat([df_buy_sell, df_tmp])


    df_buy_sell['Signal'] = df_buy_sell['Signal'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()

df_daily_buy_sell = get_daily_buy_sell()

In [13]:
df_daily_buy_sell[df_daily_buy_sell['Signal'] == 'Buy'].tail(20)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Criteria,InFolio
Date,,,,,,,,,,,,
2024-05-03 00:00:00+05:30,1545.0,1782.0,-13.30,KOTAKBANK.NS,2205.38,1545.01,0.00,42.74,28.0,Buy,X40,1.0
2024-05-14 00:00:00+05:30,1302.0,1340.0,-2.80,HCLTECH.NS,1936.35,1285.21,1.34,48.67,28.0,Buy,X40,1.0
2024-10-22 00:00:00+05:30,6678.0,6976.0,-4.27,BAJFINANCE.NS,8127.75,6279.73,6.34,21.71,28.0,Buy,X40,1.0
2024-10-31 00:00:00+05:30,3968.0,4046.0,-1.92,TCS.NS,4542.67,3534.31,12.28,14.47,29.0,Buy,X40,1.0
2024-11-07 00:00:00+05:30,3121.0,3515.0,-11.21,TITAN.NS,3854.04,3120.85,0.00,23.49,25.0,Buy,X40,-1.0
2024-11-13 00:00:00+05:30,1614.0,1718.0,-6.05,HAVELLS.NS,2082.40,1283.21,25.79,29.01,29.0,Buy,X40,NaN
2024-11-14 00:00:00+05:30,2183.0,2500.0,-12.69,NESTLEIND.NS,2755.50,2182.80,0.00,26.24,26.0,Buy,X40,-1.0
2024-11-14 00:00:00+05:30,592.0,602.0,-1.62,MARICO.NS,709.00,490.05,20.86,19.71,29.0,Buy,X40,0.0
2024-11-21 00:00:00+05:30,1223.0,1445.0,-15.36,RELIANCE.NS,1590.07,1208.74,1.18,30.01,25.0,Buy,X40,1.0


In [14]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [15]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)